In [63]:
import numpy as np
import pandas as pd
import math
import time

input_csv = 'raster.csv'
output_csv = f'Populated Raster Points{time.time()}.csv'

def combine_coordinates(x,y):
    coords=[]
    for a,b in zip(x,y):
        coords.append((a,b))
    return(coords)

def calculate_distance_matrix(coords):
    row_dist=[]
    for i in range(len(coords)):
        dist=[]
        for j in range(len(coords)):
            if j==i:
                dist.append(0)
            else:
                change_x=coords[i][0]-coords[j][0]
                change_y=coords[i][1]-coords[j][1]
                dist.append(math.sqrt(math.pow(change_x,2)+math.pow(change_y,2)))
        row_dist.append(dist)
    return(row_dist)

def calculate_tpp(dist,x,y,type_):
    tpp=[]
    ind=list(dist.sort_values().index)
    if type_=='raster':
        range_=len(dist)
    else:
        range_=int(0.25*len(dist))
    for i in range(2,range_,1):
        di=dist[ind[1]]
        dij=dist[ind[i]]
        rij=dij/di
        c_x=x[ind[i]]-x[ind[1]]
        c_y=y[ind[i]]-y[ind[1]]
        d_i_r=math.sqrt(math.pow(c_x,2)+math.pow(c_y,2))
        theta_ij=math.acos((math.pow(di,2)+math.pow(dij,2)-math.pow(d_i_r,2))/(2*di*dij))
        tpp.append((rij*math.cos(theta_ij),rij*math.sin(theta_ij)))
    return(tpp)

def to_polar_coord(delta_x,delta_y):
    rho=math.sqrt(delta_x**2+delta_y**2)
    theta=math.degrees(math.atan(delta_y/delta_x))
    return(rho,theta)

def find_candidate_cpps(tpp_r,tpp_v,delta_r_tolerance,delta_theta_tolerance):
    max_v=np.array(tpp_r).flatten().max()
    count=0
    sum_delta_r=0
    for i in (tpp_r):
        for j in (tpp_v):
            if j[0]>max_v+delta_r_tolerance:
                break
            else:
                if abs(i[0]-j[0])<=delta_r_tolerance and abs(i[1]-j[1])<=delta_theta_tolerance:
                    count=count+1
                    sum_delta_r=sum_delta_r+abs(i[0]-j[0])
    if count>=len(tpp_r):
        return(True,sum_delta_r/count)
    else:
        return(False,None)
    
raster_intersections=pd.read_csv(input_csv)
raster_intersections = raster_intersections.sample(7)
vector_intersections=pd.read_csv('vector.csv')

x_r=np.array(raster_intersections['x'])
y_r=np.array(raster_intersections['y'])

x_v=np.array(vector_intersections['x'])
y_v=np.array(vector_intersections['y'])
    
raster_coords=combine_coordinates(x_r,y_r)
vector_coords=combine_coordinates(x_v,y_v)

dist_r=calculate_distance_matrix(raster_coords)
dist_v=calculate_distance_matrix(vector_coords)

df_raster_dist=pd.DataFrame(dist_r)
df_vector_dist=pd.DataFrame(dist_v)

cpps={}
for j in range(len(df_raster_dist)):
    tpp_r=calculate_tpp(df_raster_dist[j],x_r,y_r,'raster')
    tpp_r_polar=[]
    for p in tpp_r:
        r,t=to_polar_coord(p[0],p[1])
        tpp_r_polar.append((r,t))
        
    matching_cpps=[]
    deltas_ave=[]
    for i in range(len(df_vector_dist)):
        tpp_v=calculate_tpp(df_vector_dist[i],x_v,y_v,'vector')
        tpp_v_polar=[]
        for p in tpp_v:
            r,t=to_polar_coord(p[0],p[1])
            tpp_v_polar.append((r,t))
        
        match_status,delta_=find_candidate_cpps(tpp_r_polar,tpp_v_polar,0.3,5)
        if match_status==True:
            matching_cpps.append(i)
            deltas_ave.append(delta_)
    dict_={'Index':matching_cpps,'Delta':deltas_ave}
    df=pd.DataFrame(dict_)
    optimum_id=list(df['Delta'].sort_values().index)

    if len(matching_cpps)>0:
        cpps[str(j+1)]=df['Index'][optimum_id[0]]
    else:
        cpps[str(j+1)]=None

x_coord=[]
y_coord=[]
for i in range(len(x_r)):
    if cpps[str(i+1)]!=None:
        x_coord.append(x_v[cpps[str(i+1)]])
        y_coord.append(y_v[cpps[str(i+1)]])
    else:
        x_coord.append(None)
        y_coord.append(None)
raster_intersections['Coordinate X']=x_coord
raster_intersections['Coordinate Y']=y_coord
raster_intersections.to_csv(output_csv,index=None)



print('Autogeorefencing Done')

Autogeorefencing Done


In [40]:
import numpy as np
from tps import ThinPlateSpline
import pandas as pd

tps = ThinPlateSpline(alpha=0.0)  # 0 Regularization

# Known ground truth values
p = np.array([[2126, 598], [588, 1086], [1634, 1194], [608, 451]])
i = np.array([[-9310495.04317, 4896392.15554], [-9349363.69054, 4885318.48111], [-9323285.20739,4881661.78734], [-9348299.36488, 4901172.06749]])

tps.fit(p, i)

predicted = pd.read_csv(output_csv)
predicted.dropna(subset=['Coordinate X'], inplace=True)

predicted_rasters = np.stack([predicted['x'].values, predicted['y'].values], axis=1)
predicted_vectors = np.stack([predicted['Coordinate X'].values, predicted['Coordinate X'].values], axis=1)

ground_truth_model_vectors = tps.transform(predicted_rasters)

rmse = np.linalg.norm(predicted_vectors - ground_truth_model_vectors) / np.sqrt(len(ground_truth_model_vectors))

print(rmse)

14214736.254939925


In [14]:
c = []

for i, row in raster_intersections.iterrows():
  if math.isnan(row['Coordinate X']) or math.isnan(row['Coordinate Y']):
    continue
  
  c.append([row['x'], row['y'], row['Coordinate X'], row['Coordinate Y']])

V_X, V_Y, V_XY, mo, mox, moy, [a, b, c, d] = helm_trans(np.array(c))

In [34]:
gcps = np.array(
    [[2126, 598, -9310495.04317, 4896392.15554],
    [588, 1086, -9349363.69054, 4885318.48111],
    [1634, 1194, -9323285.20739,4881661.78734],
    [608, 451, -9348299.36488, 4901172.06749]],
  )

V_X, V_Y, V_XY, mo, mox, moy, [a, b, c, d] = helm_trans(gcps)

print(mo, mox, moy)

14263.22900832384 6119.100537297428 12883.955540063658


In [55]:
command = 'gdal_translate '

for i, row in raster_intersections.iterrows():
  if math.isnan(row['Coordinate X']) or math.isnan(row['Coordinate Y']):
    continue

  command += f'-gcp {row["x"]} {row["y"]} {row["Coordinate X"]} {row["Coordinate Y"]} '

command += '-of GTiff ../data/OH-Champaign-72-01.jpg ../data/OH-Champaign-72-01-1.tif'

print(command)
os.system(command)

warp_command = f'gdalwarp -tps -r bilinear -s_srs "EPSG:3857" -t_srs "EPSG:3857" -overwrite -tr 2.0 -2.0 ../data/OH-Champaign-72-01-1.tif ../data/OH-Champaign-72-01-1-prj.tif'
print(warp_command)
# os.system(warp_command)

gdal_translate -gcp 875.0 929.0 -9333459.92016 4900016.24347 -gcp 2153.0 920.0 -9306462.49462 4870941.29055 -gcp 1634.0 1194.0 -9313871.69729 4881574.31342 -gcp 1409.0 1403.0 -9313871.69729 4881574.31342 -gcp 1893.0 1284.0 -9317142.7092 4875286.22083 -gcp 2021.0 1320.0 -9333293.05225 4902144.54607 -gcp 1338.0 1206.0 -9325295.08079 4884466.01981 -of GTiff ../data/OH-Champaign-72-01.jpg ../data/OH-Champaign-72-01-1.tif
gdalwarp -tps -r bilinear -s_srs "EPSG:3857" -t_srs "EPSG:3857" -overwrite -tr 2.0 -2.0 ../data/OH-Champaign-72-01-1.tif ../data/OH-Champaign-72-01-1-prj.tif
